In [ ]:
BERT_BASE_DIR = '/Users/m-suzuki/work/japanese-bert/jawiki-20190901/mecab-ipadic-bpe-32k/'

In [ ]:
import torch
from transformers import BertForMaskedLM
from tokenization import MecabBertTokenizer

In [ ]:
tokenizer = MecabBertTokenizer(vocab_file=f'{BERT_BASE_DIR}/vocab.txt')

In [ ]:
text = '朝食に[MASK]と牛乳を食べました。'

In [ ]:
token_ids = tokenizer.encode(text, add_special_tokens=True)

In [ ]:
token_ids

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)

In [ ]:
tokens

In [ ]:
token_ids = torch.tensor([token_ids])

In [ ]:
token_ids

In [ ]:
model = BertForMaskedLM.from_pretrained(BERT_BASE_DIR)

In [ ]:
predictions, = model(token_ids)

In [ ]:
_, top10_pred_ids = torch.topk(predictions, k=10, dim=2)

In [ ]:
top10_pred_ids

In [ ]:
for correct_id, pred_ids in zip(token_ids[0], top10_pred_ids[0]):
    correct_token = tokenizer.convert_ids_to_tokens([correct_id.item()])
    pred_tokens = tokenizer.convert_ids_to_tokens(pred_ids.tolist())
    print(correct_token, pred_tokens)
